# It looks like lucid only works with 2d 3 channel images

In [1]:
import os
import sys
import lucid
import itertools
import matplotlib

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import lucid.optvis.render as render
import tensorflow.keras.backend as K

from lucid.modelzoo.vision_models import Model
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

# sys.path.insert(0, '../clusterability_in_neural_networks/')
# from src import lucid

/Users/dwright/anaconda2/envs/local_specialization/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [2]:
tf.compat.v1.enable_eager_execution()

In [3]:
n_hidden_neurons = 256

In [4]:
i_save = tf.keras.layers.Input(shape=(28*28,3), name='input')
e1_save = tf.keras.layers.Dense(n_hidden_neurons, activation='relu', name='dense_1')(i_save)
e2_save = tf.keras.layers.Dense(n_hidden_neurons, activation='relu', name='dense_2')(e1_save)
e3_save = tf.keras.layers.Dense(n_hidden_neurons, activation='relu', name='dense_3')(e2_save)
e4_save = tf.keras.layers.Dense(n_hidden_neurons, activation='relu', name='dense_4')(e3_save)
o_save = tf.keras.layers.Dense(10, activation='softmax', name='softmax')(e4_save)
model = tf.keras.Model(inputs=i_save, outputs=o_save)

model.load_weights('/Users/dwright/dev/P3/polysemantic-neurons/experiments/mnist/alpha_1.0/trial_1/inference_model.h5')

2022-06-07 08:22:52.769253: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-06-07 08:22:52.788580: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f9ee39ca050 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-06-07 08:22:52.788605: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [5]:
# https://medium.com/@sebastingarcaacosta/how-to-export-a-tensorflow-2-x-keras-model-to-a-frozen-and-optimized-graph-39740846d9eb
frozen_out_path = '/Users/dwright/dev/P3/polysemantic-neurons/experiments/mnist/alpha_1.0/trial_1/'
frozen_graph_filename = 'inference_model_frozen_graph'

# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()

layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 60)
print("Frozen model layers: ")
for layer in layers:
    print(layer)

print("-" * 60)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)
    
# Save frozen graph to disk
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_out_path,
                  name=f"{frozen_graph_filename}.pb",
                  as_text=False)
# Save its text representation
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_out_path,
                  name=f"{frozen_graph_filename}.pbtxt",
                  as_text=True)

2022-06-07 08:22:53.065708: I tensorflow/core/grappler/devices.cc:60] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA support)
2022-06-07 08:22:53.065781: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-06-07 08:22:53.067326: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:786] Optimization results for grappler item: graph_to_optimize
2022-06-07 08:22:53.067348: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:788]   function_optimizer: function_optimizer did nothing. time = 0.006ms.
2022-06-07 08:22:53.067354: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:788]   function_optimizer: function_optimizer did nothing. time = 0.001ms.


------------------------------------------------------------
Frozen model layers: 
x
model/dense_1/MatMul/ReadVariableOp/resource
model/dense_1/MatMul/ReadVariableOp
model/dense_1/MatMul
model/dense_1/BiasAdd/ReadVariableOp/resource
model/dense_1/BiasAdd/ReadVariableOp
model/dense_1/BiasAdd
model/dense_1/Relu
model/dense_2/MatMul/ReadVariableOp/resource
model/dense_2/MatMul/ReadVariableOp
model/dense_2/MatMul
model/dense_2/BiasAdd/ReadVariableOp/resource
model/dense_2/BiasAdd/ReadVariableOp
model/dense_2/BiasAdd
model/dense_2/Relu
model/dense_3/MatMul/ReadVariableOp/resource
model/dense_3/MatMul/ReadVariableOp
model/dense_3/MatMul
model/dense_3/BiasAdd/ReadVariableOp/resource
model/dense_3/BiasAdd/ReadVariableOp
model/dense_3/BiasAdd
model/dense_3/Relu
model/dense_4/MatMul/ReadVariableOp/resource
model/dense_4/MatMul/ReadVariableOp
model/dense_4/MatMul
model/dense_4/BiasAdd/ReadVariableOp/resource
model/dense_4/BiasAdd/ReadVariableOp
model/dense_4/BiasAdd
model/dense_4/Relu
model/softm

'/Users/dwright/dev/P3/polysemantic-neurons/experiments/mnist/alpha_1.0/trial_1/inference_model_frozen_graph.pbtxt'

In [15]:
with tf.Graph().as_default() as graph, tf.Session() as sess:
    #images = tf.placeholder('float32', [None, 784], name='input')
            
    with tf.io.gfile.GFile(os.path.join(frozen_out_path, 
                                f'{frozen_graph_filename}.pb'), 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    #nodes = [n.name + ' => ' +  n.op for n in graph_def.node if n.op in ('Placeholder')]
    #tf.import_graph_def(graph_def, input_map={'^x': images})
    tf.import_graph_def(graph_def)
    print(sess.graph.get_operations())
    model = Model()
    model.load_graphdef()
    model.suggest_save_args()
    model.save(os.path.join(frozen_out_path, 'lucid_model_save.pb'),
               input_name='import/x',
               image_shape=(784,1),
               image_value_range=[0,1],
               output_names=['import/model/softmax/Softmax']
              )

[<tf.Operation 'import/x' type=Placeholder>, <tf.Operation 'import/model/dense_1/MatMul/ReadVariableOp/resource' type=Const>, <tf.Operation 'import/model/dense_1/MatMul/ReadVariableOp' type=Identity>, <tf.Operation 'import/model/dense_1/MatMul' type=MatMul>, <tf.Operation 'import/model/dense_1/BiasAdd/ReadVariableOp/resource' type=Const>, <tf.Operation 'import/model/dense_1/BiasAdd/ReadVariableOp' type=Identity>, <tf.Operation 'import/model/dense_1/BiasAdd' type=BiasAdd>, <tf.Operation 'import/model/dense_1/Relu' type=Relu>, <tf.Operation 'import/model/dense_2/MatMul/ReadVariableOp/resource' type=Const>, <tf.Operation 'import/model/dense_2/MatMul/ReadVariableOp' type=Identity>, <tf.Operation 'import/model/dense_2/MatMul' type=MatMul>, <tf.Operation 'import/model/dense_2/BiasAdd/ReadVariableOp/resource' type=Const>, <tf.Operation 'import/model/dense_2/BiasAdd/ReadVariableOp' type=Identity>, <tf.Operation 'import/model/dense_2/BiasAdd' type=BiasAdd>, <tf.Operation 'import/model/dense_2/R

In [16]:
model = Model.load(os.path.join(frozen_out_path, 'lucid_model_save.pb'))

In [17]:
render.render_vis(model, 'dense_1:0')

ValueError: Shape must be rank 2 but is rank 4 for 'import/import/model/dense_1/MatMul' (op: 'MatMul') with input shapes: [1,?,?,3], [784,256].

The above seems to imply 2d images with 3 channels.